In [2]:
from copy import deepcopy

In [82]:
class State:
    def __init__(self, board):
        self.board = board
    
    def __str__(self):
        row = len(self.board)
        col = len(self.board[0])
        
        s = ""
        
        for i in range(row):
            if i > 0:
                s += "-" * (2* col + 1)
                s += "\n"
            for j in range(col):
                if j > 0:
                    s += "|"
                if self.board[i][j] > 0:
                    s += "{:<2d}".format(self.board[i][j])
                else:
                    s += "{:<2s}".format("")
            s += "\n"
        return s
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        return str(self) == str(other)
    
    def __lt__(self, other):
        return str(self) < str(other)
    
    def __hash__(self):
        return hash(str(self))
    
    def find_cell(self, value):
        for i, row in enumerate(self.board):
            for j, val in enumerate(row):
                if val == value:
                    return (i,j)
    
    def blank(self):
        return self.find_cell(0)
        
                
    def get_heuristic_1(self, goal):
        h = 0
        for i, row in enumerate(self.board):
            for j, val in enumerate(row):
                g_i, g_j = goal.find_cell(val)
                h += abs(g_i - i) + abs(g_j - j)
        return h
    
    def get_heuristic_2(self, goal):
        h = 0
        for i, row in enumerate(self.board):
            for j, val in enumerate(row):
                g_i, g_j = goal.find_cell(val)
                diff = abs(g_i - i) + abs(g_j - j)
                if diff > 0:
                    h += 1
        return h
    
    
    def get_next(self):
        next_states = []
        
        
        num_row = len(self.board)
        num_col = len(self.board[0])
        i,j = self.blank()
        for di, dj in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            next_i = i + di
            next_j = j + dj
            if next_i < 0 or next_i >= num_row or next_j < 0 or next_j >= num_col:
                continue
            board_clone = deepcopy(self.board)
            
            board_clone[i][j], board_clone[next_i][next_j] = board_clone[next_i][next_j], board_clone[i][j]

            next_states.append(State(board_clone))
        return next_states

In [83]:
start = State([
    [1,4,3],
    [2,5,6],
    [0,8,7]
])
goal = State([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
])
start

1 |4 |3 
-------
2 |5 |6 
-------
  |8 |7 

In [56]:
goal

  |1 |2 
-------
3 |4 |5 
-------
6 |7 |8 

In [80]:
from collections import deque
from heapq import *


def bfs(start, goal):
    trace = {}
    visited = {}
    
    Q = deque()
    Q.append(start)
    found = False
    
    expand_count = 0
    
    while len(Q) > 0 and not found:
        
        cur = Q.popleft()
        visited[cur] = True
        #print("check current state ", cur)
        for next_state in cur.get_next():
            if next_state in visited:
                continue
            #print("go", cur, "->", next_state)
            trace[next_state] = cur
            Q.append(next_state)
            expand_count += 1
            if next_state == goal:
                found = True
                break
    
    path = [goal]
    cur = goal
    print("expand count", expand_count)
    
    while cur != start:
        cur = trace[cur]
        path.append(cur)
    print("path length = ", len(path))
    return path[::-1]
    

In [86]:
from collections import deque
from heapq import *


def dfs(start, goal):
    trace = {}
    visited = {}
    
    Q = deque()
    Q.append(start)
    found = False
    
    expand_count = 0
    
    while len(Q) > 0 and not found:
        
        cur = Q.pop()
        visited[cur] = True
        #print("check current state ", cur)
        for next_state in cur.get_next():
            if next_state in visited:
                continue
            #print("go", cur, "->", next_state)
            trace[next_state] = cur
            Q.append(next_state)
            expand_count += 1
            if next_state == goal:
                found = True
                break
    
    path = [goal]
    cur = goal
    print("expand count", expand_count)
    
    while cur != start:
        cur = trace[cur]
        path.append(cur)
    print("path length = ", len(path))
    return path[::-1]
    

In [87]:
path = dfs(start, goal)
print(len(path))

expand count 31857
path length =  18011
18011


In [65]:
start.get_heuristic_1(goal)

16

In [84]:
from collections import deque
from heapq import *


def a_star(start, goal):
    trace = {}
    visited = {}
    
    Q = []
    heappush(Q, (0, start))
    found = False
    
    expand_count = 0
    
    while len(Q) > 0 and not found:
        
        cost, cur = heappop(Q)
        visited[cur] = True
        #print("check current state ", cur)
        for next_state in cur.get_next():
            if next_state in visited:
                continue
            #print("go", cur, "->", next_state)
            trace[next_state] = cur
            heappush(Q, (1 + cur.get_heuristic_2(goal), next_state))
            expand_count += 1
            if next_state == goal:
                found = True
                break
    
    path = [goal]
    cur = goal
    print("expand count", expand_count)
    
    while cur != start:
        cur = trace[cur]
        path.append(cur)
    #print("path length = ", len(path))
    return path[::-1]
    

In [85]:
path = a_star(start, goal)
print(len(path))

expand count 1503
151
